# Estimating Portfolio Returns Using CAPM

### Author: Jay Parmar

#### Date: 7th Mar 2022

### Import Libraries

In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import cufflinks as cf
import matplotlib.pyplot as plt
import statsmodels.api as sm

cf.go_offline()

### Define Universe

In [3]:
# Define portfolio components
universe = ['SBIN.NS', 'TCS.NS', 'MARICO.NS', 'MARUTI.NS', 'LT.NS', 'TATASTEEL.NS']

### Download Historical Data

In [4]:
# Fetch data from Yahoo Finance
data = yf.download(universe, start='2015-1-1', end='2019-12-31')

data.head()

[*********************100%***********************]  6 of 6 completed


Adj Close                                                    \
                 LT.NS   MARICO.NS    MARUTI.NS     SBIN.NS TATASTEEL.NS   
Date                                                                       
2015-01-01  883.972046  143.839569  3115.939697  300.547516   328.333435   
2015-01-02  902.616638  146.548889  3133.520996  301.743988   333.447784   
2015-01-05  915.173828  143.883972  3215.785645  299.351074   338.278015   
2015-01-06  884.648376  145.527344  3166.771729  287.051575   321.879608   
2015-01-07  882.531067  143.017868  3206.412109  287.290894   315.709961   

                               Close                                       \
                 TCS.NS        LT.NS   MARICO.NS    MARUTI.NS     SBIN.NS   
Date                                                                        
2015-01-01  1110.821655  1001.966675  161.925003  3340.750000  314.000000   
2015-01-02  1125.615234  1023.099976  164.975006  3359.600098  315.250000   
2015-01-05  1108.509155  1037.333374  161.975006  3447.800049  312.750000   
2015-01-06  1067.642456  1002.733337  163.824997  3395.250000  299.899994   
2015-01-07  1055.031250  1000.333313  161.000000  3437.750000  300.149994   

            ...      Open                                         Volume  \
            ... MARUTI.NS     SBIN.NS TATASTEEL.NS       TCS.NS    LT.NS   
Date        ...                                                            
2015-01-01  ...   3320.25  312.450012   378.331726  1283.500000   674298   
2015-01-02  ...   3360.00  314.350006   386.811066  1275.500000  1936584   
2015-01-05  ...   3382.00  316.250000   393.480225  1290.500000  2176527   
2015-01-06  ...   3439.00  310.000000   394.432983  1264.550049  3048025   
2015-01-07  ...   3400.00  300.000000   378.998627  1235.000000  2063635   

                                                                
           MARICO.NS MARUTI.NS   SBIN.NS TATASTEEL.NS   TCS.NS  
Date                                                            
2015-01-01    142730    299232   6138488      2657081   366830  
2015-01-02    658102    233924   9935094      3604332   925740  
2015-01-05    953386    536747   9136716      5463886  1754242  
2015-01-06   1854742    422743  15329257      6476796  2423784  
2015-01-07   1531050    369241  15046745      4984692  2636332  

[5 rows x 36 columns]

In [5]:
# Extract Adj Close from the downloaded data
data = data['Adj Close']

data.head()

,LT.NS,MARICO.NS,MARUTI.NS,SBIN.NS,TATASTEEL.NS,TCS.NS
Date,,,,,,
2015-01-01,883.972046,143.839569,3115.939697,300.547516,328.333435,1110.821655
2015-01-02,902.616638,146.548889,3133.520996,301.743988,333.447784,1125.615234
2015-01-05,915.173828,143.883972,3215.785645,299.351074,338.278015,1108.509155
2015-01-06,884.648376,145.527344,3166.771729,287.051575,321.879608,1067.642456
2015-01-07,882.531067,143.017868,3206.412109,287.290894,315.709961,1055.031250


### Compute Daily Returns

In [6]:
# Compute daily percentage returns
returns_data = data.pct_change()

In [7]:
# Drop nan values
returns_data.dropna(inplace=True)

In [8]:
returns_data.head()

,LT.NS,MARICO.NS,MARUTI.NS,SBIN.NS,TATASTEEL.NS,TCS.NS
Date,,,,,,
2015-01-02,0.021092,0.018836,0.005642,0.003981,0.015577,0.013318
2015-01-05,0.013912,-0.018184,0.026253,-0.007930,0.014486,-0.015197
2015-01-06,-0.033355,0.011422,-0.015242,-0.041087,-0.048476,-0.036866
2015-01-07,-0.002393,-0.017244,0.012518,0.000834,-0.019168,-0.011812
2015-01-08,0.006365,-0.001398,0.010981,0.015659,0.015557,0.010796


### Define Weights

In [9]:
# Define weight vectors for each asset
weights = np.full(shape=(6,), fill_value=1/6)

### Compute Portfolio Returns

In [10]:
# Calculate daily portfolio returns
portfolio_returns = returns_data.dot(weights)

In [11]:
portfolio_returns

Date
2015-01-02    0.013074
2015-01-05    0.002223
2015-01-06   -0.027267
2015-01-07   -0.006211
2015-01-08    0.009660
                ...   
2019-12-23    0.000186
2019-12-24   -0.004910
2019-12-26   -0.002001
2019-12-27    0.009848
2019-12-30    0.002165
Length: 1227, dtype: float64

### Compute Portfolio Cumulative Returns

In [12]:
portfolio_cumulative_returns = (1 + portfolio_returns).cumprod() - 1

print('Total returns %.2f%%' % (portfolio_cumulative_returns[-1] * 100))

portfolio_cumulative_returns.iplot()

Total returns 88.26%


### Compute Portfolio Annualized Returns

In [13]:
# Define number of trading days
trading_days = 252

# Compute total number of data points
n = len(returns_data)

annualized_returns = ((1 + portfolio_cumulative_returns[-1]) ** (trading_days/n)) - 1

print('Annualized returns are %.2f%%' % (annualized_returns * 100))

Annualized returns are 13.87%


### Compute Portfolio Variance

In [14]:
# Compute annual covariance matrix
covar_matrix = returns_data.cov() * 252

In [15]:
covar_matrix

,LT.NS,MARICO.NS,MARUTI.NS,SBIN.NS,TATASTEEL.NS,TCS.NS
LT.NS,0.063867,0.014931,0.026786,0.041535,0.040109,0.007873
MARICO.NS,0.014931,0.062774,0.015461,0.014940,0.017625,0.003225
MARUTI.NS,0.026786,0.015461,0.064384,0.030573,0.034005,0.006388
SBIN.NS,0.041535,0.014940,0.030573,0.120567,0.048711,0.005381
TATASTEEL.NS,0.040109,0.017625,0.034005,0.048711,0.128401,0.007764
TCS.NS,0.007873,0.003225,0.006388,0.005381,0.007764,0.050889


In [16]:
# Compute annual portfolio variance
portf_annual_var = weights.T.dot(covar_matrix).dot(weights)

In [17]:
print('Annual Portfolio Variance: %.4f' % portf_annual_var)

Annual Portfolio Variance: 0.0312


In [18]:
# Alternate method
port_annual_var_2 = portfolio_returns.var() * 252

print('Annual Portfolio Variance: %.4f' % port_annual_var_2)

Annual Portfolio Variance: 0.0312


In [19]:
# Compute daily std dev
std_dev = portfolio_returns.std()

print('Portfolio Daily Standard Deviation: %.3f%%' % (std_dev * 100))

annual_std_dev = std_dev * np.sqrt(252)

print('Portfolio Annual Standard Deviation: %.3f%%' % (annual_std_dev * 100))

Portfolio Daily Standard Deviation: 1.112%
Portfolio Annual Standard Deviation: 17.650%


### Download Benchmark Data

In [20]:
# Download benchmark data which happens to be NIFTY 50 index here
nifty_data = yf.download('^NSEI', start='2015-1-1', end='2019-12-31')

[*********************100%***********************]  1 of 1 completed


In [21]:
# Compute nifty daily returns
nifty_returns = nifty_data['Adj Close'].pct_change()

In [22]:
# Drop null values
nifty_returns.dropna(inplace=True)

### Calculate Nifty Cumulative Returns

In [23]:
nifty_cumulative_returns = (1 + nifty_returns).cumprod() - 1

print('Total Nifty returns %.2f%%' % (nifty_cumulative_returns[-1] * 100))

nifty_cumulative_returns.iplot()

Total Nifty returns 45.98%


### Compute Nifty Annualized Returns

In [24]:
# Define number of trading days
trading_days = 252

# Compute total number of data points
n = len(nifty_returns)

nifty_annualized_returns = ((1 + nifty_cumulative_returns[-1]) ** (trading_days/n)) - 1

print('Nifty annualized returns are %.2f%%' % (nifty_annualized_returns * 100))

Nifty annualized returns are 8.13%


### Merge Portfolio & Benchmark Data

In [25]:
# See if portfolio & nifty have the same number of data points
print(portfolio_returns.shape)
print(nifty_returns.shape)

(1227,)
(1220,)


In [26]:
# Merge both datasets based on common dates
merged_returns = pd.merge(portfolio_returns.rename('port_returns'), nifty_returns.rename('nifty_returns'),
         right_index=True, left_index=True)

In [27]:
merged_returns

,port_returns,nifty_returns
Date,,
2015-01-05,0.002223,-0.002031
2015-01-06,-0.027267,-0.029964
2015-01-07,-0.006211,-0.003107
2015-01-08,0.009660,0.016354
2015-01-09,0.002974,0.006060
...,...,...
2019-12-23,0.000186,-0.000737
2019-12-24,-0.004910,-0.003931
2019-12-26,-0.002001,-0.007205


In [28]:
merged_returns.cov() * 252

,port_returns,nifty_returns
port_returns,0.031196,0.020460
nifty_returns,0.020460,0.018726


### Calculate Beta

###### Using formula

In [29]:
# Compute annual variance
beta = (merged_returns.cov() * 252).iloc[0,1] / (nifty_returns.var() * 252)

In [30]:
print('Portfolio Beta: %.2f' % beta)

Portfolio Beta: 1.09


###### Using regression

In [31]:
# Define dependent variable
y = merged_returns.pop('port_returns')

In [32]:
# Define independent variable
X = sm.add_constant(merged_returns)

In [33]:
# Regress and find the coefficient
regress = sm.OLS(y, X).fit()

print(regress.summary())

                            OLS Regression Results                            
Dep. Variable:           port_returns   R-squared:                       0.717
Model:                            OLS   Adj. R-squared:                  0.716
Method:                 Least Squares   F-statistic:                     3080.
Date:                Mon, 07 Mar 2022   Prob (F-statistic):               0.00
Time:                        16:41:47   Log-Likelihood:                 4526.6
No. Observations:                1220   AIC:                            -9049.
Df Residuals:                    1218   BIC:                            -9039.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0002      0.000      1.064

### Estimate Portfolio Returns using CAPM

In [34]:
# Annual risk free rate
risk_free_rate = 0.00

In [35]:
# Estimate portfolio returns using the CAPM
expected_portfolio_returns = risk_free_rate + beta * (nifty_annualized_returns  - risk_free_rate)

In [36]:
print('Expected Portfolio Annual Returns: %.3f%%' % (expected_portfolio_returns * 100))

Expected Portfolio Annual Returns: 8.880%
